## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


Possible questions

Does area income effect price of houses

Does property size effect housing price

Does Age of property effect price

In [513]:
#find lat and long of city center then subtract out lat and long form it to find the difference 
# weak correlation to test in model Floor waterfront view and grade 

In [514]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
from datetime import datetime
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn import metrics
from scipy import stats
import math
from PIL import Image

In [515]:
df = pd.read_csv("data/kc_house_data_no_miss_values.csv")
df =df.drop(["Unnamed: 0"], axis = 1)

In [516]:
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Income,house_age,year_sold,season,renovated
0,221900.0,3,1.00,1180,5650,1.0,0,4,0,8,1180,0.0,1955,0.0,66,47.5112,-122.257,1340,5650,113555.0,67,2014,3,0
1,205425.0,2,1.00,880,6780,1.0,0,4,2,7,880,0.0,1945,0.0,66,47.5009,-122.248,1190,6780,113555.0,77,2014,1,0
2,445000.0,3,2.25,2100,8201,1.0,0,0,0,9,1620,480.0,1967,0.0,66,47.5091,-122.244,2660,8712,113555.0,55,2014,2,0
3,236000.0,3,1.00,1300,5898,1.0,0,4,0,8,1300,0.0,1961,0.0,66,47.5053,-122.255,1320,7619,113555.0,61,2014,2,0
4,170000.0,2,1.00,860,5265,1.0,0,4,0,7,860,0.0,1931,0.0,66,47.5048,-122.272,1650,8775,113555.0,91,2015,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21415,2720000.0,4,3.25,3990,18115,2.0,0,4,2,1,3990,0.0,1989,0.0,24,47.6177,-122.229,3450,16087,7190.0,33,2014,3,0
21416,1950000.0,4,3.50,4065,18713,2.0,0,4,2,0,4065,0.0,1987,0.0,24,47.6209,-122.237,3070,18713,7190.0,35,2014,2,0
21417,2250000.0,5,5.25,3410,8118,2.0,0,4,0,1,3410,0.0,2006,0.0,24,47.6295,-122.236,3410,16236,7190.0,16,2015,1,0
21418,2900000.0,5,4.00,5190,14600,2.0,0,2,0,1,5190,0.0,2013,0.0,24,47.6102,-122.225,3840,19250,7190.0,9,2014,3,0


Initial Model

Possibly drop

br : sqft liv

sqft liv 15 : sqft liv

sqft above : sqft liv

sqft lot 15 : sqft lot



Keep sqft_living, drop sqft_above, sqft_living15 due to high collinearity 

Keep sqft_lot drop sqft_lot15

df = df.drop(["sqft_above", "sqft_living15", "sqft_lot15", "lat", "long" ], axis=1)

Although the above table shows that there is a high collinearity between bedrooms and sqft_living, we believe that the number of bedrooms will play a key role in the price of the home, will revisit after modeling to verify if we need

In [517]:
X = df.drop("price", axis=1)
y= df["price"]

In [518]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [519]:
X_train.dtypes

bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement    float64
yr_built           int64
yr_renovated     float64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
Income           float64
house_age          int64
year_sold          int64
season             int64
renovated          int64
dtype: object

In [520]:
X_train = X_train.apply(pd.to_numeric)

In [521]:
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Income,house_age,year_sold,season,renovated
13513,4,3.00,4750,21701,1.5,0,4,4,1,4750,0.0,1976,0.0,3,47.6454,-122.218,3120,18551,127274.0,46,2014,3,0
13510,3,1.75,1640,18730,1.5,0,4,2,8,1640,0.0,1946,0.0,3,47.5975,-122.196,1670,12154,127274.0,76,2014,2,0
5673,4,2.25,2115,6234,2.0,0,4,0,8,2115,0.0,1998,0.0,1,47.2850,-122.201,2440,6366,121182.0,24,2014,2,0
15291,3,2.25,2220,4060,1.0,0,4,0,8,1330,890.0,1993,0.0,52,47.5188,-122.265,1930,5625,208488.0,29,2014,2,0
17139,3,1.75,1530,8028,1.0,0,4,0,8,1200,330.0,1967,0.0,65,47.7768,-122.361,1530,8028,82472.0,55,2015,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,3,1.75,1590,2500,1.5,0,4,2,8,1190,400.0,1908,0.0,43,47.6668,-122.299,1420,3800,108736.0,114,2014,3,0
11964,3,2.50,1780,5015,2.0,0,4,0,8,1780,0.0,2010,0.0,26,47.3352,-122.052,2010,5250,217292.0,12,2015,1,0
5390,2,1.75,860,5535,1.0,0,4,0,8,860,0.0,1948,0.0,42,47.6901,-122.331,1420,4960,277894.0,74,2014,3,0
860,4,2.00,2510,9750,1.0,0,4,0,9,1630,880.0,1960,0.0,16,47.7438,-122.229,1980,9750,90167.0,62,2015,1,0


In [522]:
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler

# Importances are based on coefficient magnitude, so
# we need to scale the data to normalize the coefficients
X_train_for_RFECV = StandardScaler().fit_transform(X_train)

model_for_RFECV = LinearRegression()

# Instantiate and fit the selector
selector = RFECV(model_for_RFECV, min_features_to_select=10 ,cv=splitter)
selector.fit(X_train, y_train)

# Print the results
print("Was the column selected?")
for index, col in enumerate(X_train.columns):
    print(f"{col}: {selector.support_[index]}")
####updated # of features

Was the column selected?
bedrooms: True
bathrooms: True
sqft_living: True
sqft_lot: False
floors: True
waterfront: True
view: True
condition: True
grade: True
sqft_above: True
sqft_basement: True
yr_built: True
yr_renovated: True
zipcode: True
lat: True
long: True
sqft_living15: True
sqft_lot15: True
Income: True
house_age: True
year_sold: True
season: True
renovated: True


In [523]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(17136, 23)
(4284, 23)
(17136,)
(4284,)


In [524]:
X_train = df.drop("price", axis=1)
y_train = df["price"]

X_test = df.drop("price", axis=1)
y_test = df["price"]

X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Income,house_age,year_sold,season,renovated
0,3,1.00,1180,5650,1.0,0,4,0,8,1180,0.0,1955,0.0,66,47.5112,-122.257,1340,5650,113555.0,67,2014,3,0
1,2,1.00,880,6780,1.0,0,4,2,7,880,0.0,1945,0.0,66,47.5009,-122.248,1190,6780,113555.0,77,2014,1,0
2,3,2.25,2100,8201,1.0,0,0,0,9,1620,480.0,1967,0.0,66,47.5091,-122.244,2660,8712,113555.0,55,2014,2,0
3,3,1.00,1300,5898,1.0,0,4,0,8,1300,0.0,1961,0.0,66,47.5053,-122.255,1320,7619,113555.0,61,2014,2,0
4,2,1.00,860,5265,1.0,0,4,0,7,860,0.0,1931,0.0,66,47.5048,-122.272,1650,8775,113555.0,91,2015,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21415,4,3.25,3990,18115,2.0,0,4,2,1,3990,0.0,1989,0.0,24,47.6177,-122.229,3450,16087,7190.0,33,2014,3,0
21416,4,3.50,4065,18713,2.0,0,4,2,0,4065,0.0,1987,0.0,24,47.6209,-122.237,3070,18713,7190.0,35,2014,2,0
21417,5,5.25,3410,8118,2.0,0,4,0,1,3410,0.0,2006,0.0,24,47.6295,-122.236,3410,16236,7190.0,16,2015,1,0
21418,5,4.00,5190,14600,2.0,0,2,0,1,5190,0.0,2013,0.0,24,47.6102,-122.225,3840,19250,7190.0,9,2014,3,0


In [525]:
baseline_model = LinearRegression()

splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

baseline_scores = cross_validate(
    estimator=baseline_model,
    X=X_train[["sqft_living"]],
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Train score:     ", baseline_scores["train_score"].mean())
print("Validation score:", baseline_scores["test_score"].mean())

Train score:      0.48968535742377295
Validation score: 0.49653373354233493


In [486]:
df_numerical = df[['price', 'bedrooms','bathrooms', 'sqft_living', 'floors', 'sqft_lot', 'sqft_basement', 'sqft_living15', 'sqft_lot15','Income',]].copy()

In [487]:
df_numerical1 = df[['price', 'bedrooms','bathrooms', 'sqft_living', 'floors', 'sqft_lot', 'sqft_basement', 'Income',]].copy()

In [488]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_numerical.values, i) for i in range(df_numerical.shape[1])]
vif["features"] = df_numerical.columns
vif.round(2)

,VIF Factor,features
0,6.60,price
1,18.72,bedrooms
2,24.59,bathrooms
3,33.17,sqft_living
4,13.75,floors
5,2.35,sqft_lot
6,2.40,sqft_basement
7,19.79,sqft_living15
8,2.57,sqft_lot15
9,5.63,Income


Checking Data

In [489]:
y = df["price"]
X = df.drop("price", axis=1)

In [490]:
X.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'Income', 'house_age', 'year_sold',
       'season', 'renovated'],
      dtype='object')

In [491]:
pd.set_option('display.max_columns', None)
X

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Income,house_age,year_sold,season,renovated
0,3,1.00,1180,5650,1.0,0,4,0,8,1180,0.0,1955,0.0,66,47.5112,-122.257,1340,5650,113555.0,67,2014,3,0
1,2,1.00,880,6780,1.0,0,4,2,7,880,0.0,1945,0.0,66,47.5009,-122.248,1190,6780,113555.0,77,2014,1,0
2,3,2.25,2100,8201,1.0,0,0,0,9,1620,480.0,1967,0.0,66,47.5091,-122.244,2660,8712,113555.0,55,2014,2,0
3,3,1.00,1300,5898,1.0,0,4,0,8,1300,0.0,1961,0.0,66,47.5053,-122.255,1320,7619,113555.0,61,2014,2,0
4,2,1.00,860,5265,1.0,0,4,0,7,860,0.0,1931,0.0,66,47.5048,-122.272,1650,8775,113555.0,91,2015,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21415,4,3.25,3990,18115,2.0,0,4,2,1,3990,0.0,1989,0.0,24,47.6177,-122.229,3450,16087,7190.0,33,2014,3,0
21416,4,3.50,4065,18713,2.0,0,4,2,0,4065,0.0,1987,0.0,24,47.6209,-122.237,3070,18713,7190.0,35,2014,2,0
21417,5,5.25,3410,8118,2.0,0,4,0,1,3410,0.0,2006,0.0,24,47.6295,-122.236,3410,16236,7190.0,16,2015,1,0
21418,5,4.00,5190,14600,2.0,0,2,0,1,5190,0.0,2013,0.0,24,47.6102,-122.225,3840,19250,7190.0,9,2014,3,0


In [492]:
y

0         221900.0
1         205425.0
2         445000.0
3         236000.0
4         170000.0
           ...    
21415    2720000.0
21416    1950000.0
21417    2250000.0
21418    2900000.0
21419    1870000.0
Name: price, Length: 21420, dtype: float64

In [493]:
model = sm.OLS(y, sm.add_constant(X), missing = "drop").fit()
results = model.summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.685
Method:                 Least Squares   F-statistic:                     2117.
Date:                Wed, 05 Jan 2022   Prob (F-statistic):               0.00
Time:                        20:58:33   Log-Likelihood:            -2.9251e+05
No. Observations:               21420   AIC:                         5.851e+05
Df Residuals:                   21397   BIC:                         5.852e+05
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -69.2842      4.541    -15.257

In [494]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(21420, 23)
(21420, 23)
(21420,)
(21420,)


In [495]:
data_transformation = preprocessing.StandardScaler()
data = data_transformation.fit_transform(X_train)
data

array([[-0.41257996, -1.45495786, -0.98296196, ..., -0.69429479,
         1.42849868, -0.18916492],
       [-1.52003144, -1.45495786, -1.30947943, ..., -0.69429479,
        -0.5038921 , -0.18916492],
       [-0.41257996,  0.1711599 ,  0.0183583 , ..., -0.69429479,
         0.46230329, -0.18916492],
       ...,
       [ 1.80232299,  4.07384254,  1.44415128, ...,  1.4403104 ,
        -0.5038921 , -0.18916492],
       [ 1.80232299,  2.44772478,  3.3814883 , ..., -0.69429479,
         1.42849868, -0.18916492],
       [ 0.69487152,  2.12250122,  1.85774008, ...,  1.4403104 ,
        -1.47008748, -0.18916492]])

In [496]:
df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Income,house_age,year_sold,season,renovated
0,221900.0,3,1.00,1180,5650,1.0,0,4,0,8,1180,0.0,1955,0.0,66,47.5112,-122.257,1340,5650,113555.0,67,2014,3,0
1,205425.0,2,1.00,880,6780,1.0,0,4,2,7,880,0.0,1945,0.0,66,47.5009,-122.248,1190,6780,113555.0,77,2014,1,0
2,445000.0,3,2.25,2100,8201,1.0,0,0,0,9,1620,480.0,1967,0.0,66,47.5091,-122.244,2660,8712,113555.0,55,2014,2,0
3,236000.0,3,1.00,1300,5898,1.0,0,4,0,8,1300,0.0,1961,0.0,66,47.5053,-122.255,1320,7619,113555.0,61,2014,2,0
4,170000.0,2,1.00,860,5265,1.0,0,4,0,7,860,0.0,1931,0.0,66,47.5048,-122.272,1650,8775,113555.0,91,2015,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21415,2720000.0,4,3.25,3990,18115,2.0,0,4,2,1,3990,0.0,1989,0.0,24,47.6177,-122.229,3450,16087,7190.0,33,2014,3,0
21416,1950000.0,4,3.50,4065,18713,2.0,0,4,2,0,4065,0.0,1987,0.0,24,47.6209,-122.237,3070,18713,7190.0,35,2014,2,0
21417,2250000.0,5,5.25,3410,8118,2.0,0,4,0,1,3410,0.0,2006,0.0,24,47.6295,-122.236,3410,16236,7190.0,16,2015,1,0
21418,2900000.0,5,4.00,5190,14600,2.0,0,2,0,1,5190,0.0,2013,0.0,24,47.6102,-122.225,3840,19250,7190.0,9,2014,3,0


In [497]:
df2 = df.to_numpy()

In [498]:
y1 = df2[:, 0]
y1

array([ 221900.,  205425.,  445000., ..., 2250000., 2900000., 1870000.])

In [499]:
X1 = df2[:,1:]
X1

array([[3.000e+00, 1.000e+00, 1.180e+03, ..., 2.014e+03, 3.000e+00,
        0.000e+00],
       [2.000e+00, 1.000e+00, 8.800e+02, ..., 2.014e+03, 1.000e+00,
        0.000e+00],
       [3.000e+00, 2.250e+00, 2.100e+03, ..., 2.014e+03, 2.000e+00,
        0.000e+00],
       ...,
       [5.000e+00, 5.250e+00, 3.410e+03, ..., 2.015e+03, 1.000e+00,
        0.000e+00],
       [5.000e+00, 4.000e+00, 5.190e+03, ..., 2.014e+03, 3.000e+00,
        0.000e+00],
       [4.000e+00, 3.750e+00, 3.790e+03, ..., 2.015e+03, 0.000e+00,
        0.000e+00]])

In [500]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

In [501]:
print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

(17136, 23)
(4284, 23)
(17136,)
(4284,)


In [502]:
data_transformation = preprocessing.StandardScaler()
data = data_transformation.fit_transform(X_train1)

In [503]:
model = LinearRegression().fit(data, y_train1)
print(model.intercept_)
print(model.coef_)

543071.2926004977
[  -42356.56800406    45322.22937036   142182.08662029     3616.24526958
    19475.08679572    57478.41303805   -27944.54876168    21812.82678695
   -44747.45367041    59029.83616029    19129.1337642    -27287.22727651
  1688380.62314374   -38358.40959526    94117.4920913    -51022.91011271
    42263.77966809   -12609.39829189   -22534.55292632    27287.22727651
    15522.65021374     3864.28715916 -1674709.5196107 ]


In [504]:
accuracy = model.score(data, y_train1)
accuracy

0.6877809252590406

In [505]:
test_accuracy = model.score(data_transformation.transform((X_test1)), y_test1)
test_accuracy

0.6722761250554701

In [506]:
y_pred = model.predict(data_transformation.transform(X_test1))

In [507]:
y_pred

array([472499.69184523, 446820.35440406, 487883.08315226, ...,
       325643.89832322, 356966.73162098, 464080.71945247])

In [508]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test1, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test1, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test1, y_pred)))

Mean Absolute Error: 130462.02300234168
Mean Squared Error: 41027049814.25112
Root Mean Squared Error: 202551.35105511177


In [509]:
model1 = pd.DataFrame({'Actual value': y_test1, 'Predicted value': y_pred, "Difference" : (y_test1-y_pred).round(2)})
model1.head()


,Actual value,Predicted value,Difference
0,225000.0,472499.691845,-247499.69
1,610000.0,446820.354404,163179.65
2,482500.0,487883.083152,-5383.08
3,390000.0,261195.248805,128804.75
4,380000.0,287218.936263,92781.06


In [510]:
model1["Difference"].mean()

-683.47216153128

In [511]:
MSE = np.square(np.subtract(y_test1,y_pred)).mean() 
 
RMSE = math.sqrt(MSE)
RMSE


202551.35105511177